# Prototypy Danych

![Prototyping Data](https://live.staticflickr.com/65535/54547003811_0b48d94df0_b.jpg)

*Grafika wygenerowana przez Sora - ChatGPT.*

## Wstęp

Modele uczenia maszynowego mogą analizować skomplikowane dane wejściowe takie jak obrazy, tekst po przekształceniu ich do wielowymiarowych reprezentacji zwanych zanurzeniami (ang. *embeddings*). W takiej przestrzeni wielowymiarowej obiekty o zbliżonych cechach umieszczane są blisko siebie, co umożliwia modelom skuteczne analizowanie relacji i podobieństw między nimi oraz podejmowanie trafnych decyzji.

Trenowanie coraz bardziej nowoczesnych i większych modeli wymaga zwiększonej liczby danych, co wiąże się ze zwiększonym zapotrzebowaniem na zasoby – zarówno pamięć, jak i moc obliczeniową. Aby ograniczyć te koszty, poszukuje się metod umożliwiających redukcję danych przy zachowaniu ich informacyjnej wartości. W takich przypadkach celem jest więc stworzenie mniejszego zbioru danych, który pozwoli na wytrenowanie modelu o podobnej jakości co na oryginalnych danych. 

## Zadanie

To zadanie stanowi modyfikację podejścia opisanego we wstępie. Twoim zadaniem będzie wyznaczenie małego zbioru na podstawie dostarczonego zbioru treningowego i dostarczonego klasyfikatora. Elementy z nowo utworzonego zbioru będziemy nazywać prototypami. Co ważne, Twoim zadaniem jest stworzenie zbioru 150 zanurzeń (prototypów) na podstawie reprezentacji z ostatniej warstwy sieci *Net()*. Oznacza to, że dla każdego obrazka ze zbioru testowego zostanie przypisane jego wyjście z ostatniej warstwy sieci *Net()* (zanurzenie).

### Ewaluacja

Każdej próbce ze zbioru testowego zostanie przypisana klasa prototypu, który znajduje się najbliżej (w sensie metryki euklidesowej) w przestrzeni zanurzeń (algorytm one-nearest-neighbor). Ocenie podlegać będzie poprawność klasyfikacji.

### Kryterium Oceny

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów.
Twoje rozwiązanie będzie oceniane na podstawie dokładności klasyfikacji (*acc*) uzyskanej na tajnym zbiorze testowym.

Liczba punktów, jaką otrzymasz, będzie liczona według wzoru: 
$$
\text{punkty} =
\begin{cases}
0 & \text{gdy } acc \leq 0.40 \\
100 \cdot \frac{acc - 0.4}{0.68-0.40} & \text{gdy } 0.40 < acc < 0.68 \\
100 & \text{gdy } acc \geq 0.68 
\end{cases}
$$

### Ograniczenia
- Do utworzenia zbioru prototypów możesz wykorzystać zanurzenia zbioru zdjęć ze zbioru treningowego (`train_embeddings`), oraz etykiet klas dla tego zbioru `train_labels`.
- Twoje rozwiązanie będzie testowane na Platformie Konkursowej bez dostępu do internetu oraz w środowisku z GPU.
- Ewaluacja Twojego finalnego rozwiązania na Platformie Konkursowej nie może trwać dłużej niż 5 minut z GPU.
- Możesz używać następujących bibliotek:
    - `random`
    - `numpy`
    - `math`
    - `torch`
    - `sklearn`

## Pliki zgłoszeniowe
Ten notebook uzupełniony o Twoje rozwiązanie zawierające implementacje klasy `YourSolution`

# Kod Startowy
W tej sekcji inicjalizujemy środowisko poprzez zaimportowanie potrzebnych bibliotek i funkcji. Przygotowany kod ułatwi Tobie efektywne operowanie na danych i budowanie właściwego rozwiązania.

In [1]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

FINAL_EVALUATION_MODE = False  # Podczas sprawdzania ustawimy tą flagę na True.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
# Poniżej zaimportowane są biblioteki potrzebne do wykonania tego notebooka.
# W swoim rozwiązaniu będziesz mógł korzystać z innych bibliotek wymienionych
# wyżej. Miejsce na ich zaimportowanie znajduje się poniżej w sekcji
# na Twoje rozwiązanie.

import os
import torch
import pickle
import numpy as np
import math
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.models import mobilenetv3, MobileNet_V3_Small_Weights
import torchinfo
import tarfile
from tqdm import tqdm, trange
from typing import Tuple, Type

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
if not FINAL_EVALUATION_MODE:
    import gdown
    import os

    GDRIVE_DATA = [
        ("15v0YZx6frCzdVT9zPcoD7Vr-N0mzFKsN", "models/mobilenet_v3_small.pth"),
        ("1gHeU8Of1BYbrFWG3Roi1esYq_pevhSAo", "data/val_embeddings.pkl"),
        ("1p0CHP5APgjx18Iq1Fj5rKq9nRt9ZlP_X", "data/Cifar100/train_val.gz")
    ]
    
    for file_id, output in GDRIVE_DATA:        
        url = f'https://drive.google.com/uc?id={file_id}'
        os.makedirs(os.path.dirname(output), exist_ok=True)
        gdown.download(url, output, quiet=False)
        
        print(f"Downloaded: {output}")

## Architektura Modelu

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
class Net(nn.Module):
    """
    Jest to klasa reprezentująca klasyfikator, który zostanie użyty do
    przekształcenia obrazków do ich zanurzeń (ang. embeddingów).
    """
    def __init__(self, verbose=False):
        super().__init__()

        self.net = self.get_net().to(DEVICE)

        self.IMAGE_HEIGHT = 224
        self.IMAGE_WIDTH = 224
        self.INPUT_CHANNELS = 3

        if verbose:
            self.summary()

    @staticmethod
    def get_net() -> nn.Module:
        mobilenet = mobilenetv3.mobilenet_v3_small(weights=None)
        mobilenet.load_state_dict(torch.load('models/mobilenet_v3_small.pth'))
        return mobilenet

    def summary(self) -> None:
        """
        Wypisuje opis i podsumowanie sieci za pomocą biblioteki 
        torchinfo. 
        """
        print(
            torchinfo.summary(
                self.net,
                input_size=(
                    1,
                    self.INPUT_CHANNELS,
                    self.IMAGE_HEIGHT,
                    self.IMAGE_WIDTH,
                ),
            )
        )

    def get_embedding(self, x: torch.Tensor) -> torch.Tensor: 
        """
        Dla zadanego obrazka `x` zwraca jego embedding
        """
        x = self.net(x)
        return torch.flatten(x, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

In [ ]:
# Inicjacja sieci
model = Net(verbose=True)

## Ładowanie danych

In [6]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
class Dataset:
    """
    Klasa reprezentująca dataset CIFAR-100. Zbiór ten jest podzielony na dokładnie
    100 klas. Klasa udostępnia metody `get_trian_loader` i `get_val_loader`.
    """
    def __init__(self, train_dataset, val_dataset, verbose: bool = False):
        self.dataset_name = "CIFAR-100"
        self.mean = (0.485, 0.456, 0.406)
        self.std = (0.229, 0.224, 0.225)
        self.NUM_OF_CLASSES = 100

        self.train_dataset = train_dataset

        self.val_dataset = val_dataset

        if verbose:
            self.summary()

    def get_train_loader(self, batch_size: int = 128) -> DataLoader:
        """Zwraca loader danych treningowych."""
        return DataLoader(self.train_dataset, batch_size=batch_size, shuffle=True)

    def get_val_loader(self, batch_size: int = 128) -> DataLoader:
        """Zwraca loader danych walidacyjnych."""
        return DataLoader(self.val_dataset, batch_size=batch_size, shuffle=False)

    def summary(self, show_image: bool = True):
        """Wyświetla krótkie podsumowanie zbioru danych i przykładowy obrazek."""
        print(f"\n📊 Summary of {self.dataset_name} Dataset")
        print(f"🔢 Number of classes: {self.NUM_OF_CLASSES}")
        print(f"📁 Number of training examples: {len(self.train_dataset)}")
        print(f"📁 Number of validation examples: {len(self.val_dataset)}")
        print(
            f"📦 Examples per class in train (approx): {len(self.train_dataset) // self.NUM_OF_CLASSES}"
        )
        print(
            f"📦 Examples per class in val (approx): {len(self.val_dataset) // self.NUM_OF_CLASSES}"
        )

        image, label = self.train_dataset[2]
        print(f"🖼️ Example image shape: {image.shape}")
        print(f"🏷️ Example label: {label} ({self.train_dataset.classes[label]})")

        if show_image:
            img_np = image.permute(1, 2, 0)
            # unnormalize
            img_np = img_np * torch.tensor(self.std) + torch.tensor(self.mean)
            img_np = img_np.clip(0, 1).numpy()

            plt.imshow(img_np)
            plt.title(f"Class: {label}")
            plt.axis("off")
            plt.show()

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
READONLY_DATA_PATH = "data/Cifar100"
OUTPUT_PATH = "file_output"
os.makedirs(OUTPUT_PATH, exist_ok=True)
def unpack_tar_gz(filename: str, path: str = OUTPUT_PATH) -> None:
    """ Rozpakowuje archiwum tar.gz """
    with tarfile.open(filename, "r:gz") as tar:
        tar.extractall(path=path)
        print(f"Extracted {filename} to {path}")


def load_dataset_from_pkl(filename: str) -> Dataset:
    """ Wczytuje zbiór danych z pliku .pkl """
    with open(filename, "rb") as f:
        dataset = pickle.load(f)
    return dataset

Ta komórka używa wcześniej zdefiniowanych funkcji, aby załadować zbiór treningowy i walidacyjny.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

unpack_tar_gz(
    os.path.join(READONLY_DATA_PATH, "train_val.gz"), os.path.join(OUTPUT_PATH)
)


loaded_data = load_dataset_from_pkl(os.path.join(OUTPUT_PATH, "train_val.pkl"))

# Wyodrębnij zbiór treningowy i walidacyjny
train_dataset = loaded_data["train"]
val_dataset = loaded_data["val"]
classes = loaded_data["classes"]
train_dataset.classes = classes
val_dataset.classes = classes

Tu inicjowany jest obiekt typu `Dataset`, aby wczytać obiekty typu Dataloader odpowiednio dla zbioru treningowego i walidacyjnego. Wypisywane jest też podsumowanie zbioru danych (patrz wynik komórki poniżej) 

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI #########################

ds = Dataset(train_dataset, val_dataset, verbose=True)

BATCH_SIZE = 128
train_loader = ds.get_train_loader(BATCH_SIZE)
val_loader = ds.get_val_loader(BATCH_SIZE)

## Obliczenie zanurzeń danych
Od teraz będziesz pracować na zanurzeniach danych, które są reprezentacjami zdjęć. Dla komfortu pracy, od razu po policzeniu, zanurzenia zapisywane są na dysku, abyś nie musiał ich obliczać za każdym razem, gdy uruchomisz na nowo notatnik. Wynikiem tej komórki są zmienne `train_embeddings`, `val_embeddings`, `train_labels`, `val_labels`. Są to obiekty typu `torch.tensor` zawierające dane testowe i walidacyjne, z których będziesz korzystać w dalszej części notatnika i w swoim rozwiązaniu. 

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def get_embeddings(model, dataloader: DataLoader) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Wyodrębnia embeddingi i etykiety z modelu dla danych z dataloadera.
    """
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for images, batch_labels in tqdm(dataloader):
            images, batch_labels = images.to(DEVICE), batch_labels.to(DEVICE)
            batch_embeddings = model.get_embedding(images)
            embeddings.append(batch_embeddings)
            labels.append(batch_labels)
    return torch.cat(embeddings), torch.cat(labels)


def save_embeddings(embeddings: torch.Tensor, labels: torch.Tensor, filename: str) -> None:
    """
    Zapisuje embeddingi i etykiety do pliku.
    """
    data = {"embeddings": embeddings, "labels": labels}
    with open(filename, "wb") as f:
        pickle.dump(data, f)


def load_embeddings(filename: str) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Ładuje embeddingi i etykiety z pliku.
    """
    with open(filename, "rb") as f:
        data = pickle.load(f)
    return data["embeddings"], data["labels"]


embedding_path = os.path.join(OUTPUT_PATH, "train_embeddings.pkl")
val_embedding_path = os.path.join(OUTPUT_PATH, "val_embeddings.pkl")

# Oblicz embeddingi lub wczytaj wcześniej policzone z pliku
if os.path.exists(embedding_path):
    print(f"Loading cached training embeddings from: {embedding_path}")
    train_embeddings, train_labels = load_embeddings(embedding_path)
else:
    print("Generating new training embeddings...")
    train_embeddings, train_labels = get_embeddings(model, train_loader)
    save_embeddings(train_embeddings, train_labels, embedding_path)
    print(f"Saved training embeddings to: {embedding_path}")

if os.path.exists(val_embedding_path):
    print(f"Loading cached validation embeddings from: {val_embedding_path}")
    val_embeddings, val_labels = load_embeddings(val_embedding_path)
else:
    print("Generating new validation embeddings...")
    val_embeddings, val_labels = get_embeddings(model, val_loader)
    save_embeddings(val_embeddings, val_labels, val_embedding_path)
    print(f"Saved validation embeddings to: {val_embedding_path}")

train_embeddings = train_embeddings.to(DEVICE)
val_embeddings = val_embeddings.to(DEVICE)

train_labels = train_labels.to(DEVICE)
val_labels = val_labels.to(DEVICE)

## Kod z kryterium oceniającym
Poniższy kod będzie używany do oceny rozwiązania na zbiorze testowym. Jedyną zmianą będzie podmiana zbioru walidacyjnego na tajny zbiór testowy. Możesz tej klasy użyć do sprawdzania swojego rozwiązania na zbiorze walidacyjnym dołączonym do zadania. 

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def accuracy(
    prototypes: torch.Tensor,
    prototypes_labels: torch.Tensor,
    embeddings_val: torch.Tensor,
    labels_val: torch.Tensor
) -> float:
    """
    Oblicza dokładność klasyfikacji na podstawie najbliższych prototypów.
    """
    dist = torch.cdist(embeddings_val, prototypes)
    idx = dist.argmin(dim=1).to(prototypes.device)
    predictions = prototypes_labels[idx]
    predicted_class = predictions.clone().detach().to(labels_val.device)
    return (predicted_class == labels_val).to(torch.float32).mean().item()


class SolutionHolder():    
    """
    Klasa odpowiedzialna za rejestrowanie i ocenę rozwiązań prototypowych
    w oparciu o dane embeddingów i etykiet.
    """
    def __init__(
        self,
        train_embeddings: torch.Tensor,
        train_labels: torch.Tensor,
        val_embeddings: torch.Tensor,
        val_labels: torch.Tensor
    ):
        self.train_embeddings = train_embeddings
        self.train_labels = train_labels
        self.val_embeddings = val_embeddings
        self.val_labels = val_labels
        self.sol_list = []
    
    def assert_solution(self, prototypes: torch.Tensor, prototypes_labels: torch.Tensor):
        """
        Sprawdza, czy rozwiązanie spełnia wymagane warunki.
        """
        # Stałe wymagane warunki
        REQUIRED_NUM_PROTOTYPES = 150
        MAX_LABEL = 100
        DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Sprawdzenie typów
        assert isinstance(prototypes, torch.Tensor), "prototypes musi być typu torch.Tensor"
        assert isinstance(prototypes_labels, torch.Tensor), "prototypes_labels musi być typu torch.Tensor"

        # Sprawdzenie typu danych
        assert prototypes.dtype == torch.float32, "prototypes.dtype musi być torch.float32"
        assert prototypes_labels.dtype == torch.long, "prototypes_labels.dtype musi być torch.long"

        # Sprawdzenie kształtów
        assert prototypes.shape[0] == REQUIRED_NUM_PROTOTYPES, f"prototypes powinien mieć 150 wierszy, otrzymano {prototypes.shape[0]}"
        assert prototypes_labels.shape[0] == REQUIRED_NUM_PROTOTYPES, f"prototypes_labels powinien mieć 150 elementów, otrzymano {prototypes_labels.shape[0]}"
        assert len(prototypes_labels.shape) == 1, "prototypes_labels musi być wektorem 1D"

        # Sprawdzenie wartości etykiet
        assert torch.all((prototypes_labels >= 0) & (prototypes_labels < MAX_LABEL)), "Wszystkie etykiety w prototypes_labels muszą być w zakresie [0, 100)"

    
    def register_class_solution(self, solution_class: Type, name: str) -> None:
        """
        Rejestruje rozwiązanie na podstawie klasy zawierającej metodę get_prototypes.
        """
        solution = solution_class()
        prototypes, prototypes_labels = solution.get_prototypes(self.train_embeddings, self.train_labels)
        self.assert_solution(prototypes, prototypes_labels)
        self.register_solution(prototypes, prototypes_labels, name)

    def register_solution(self, prototypes: torch.Tensor, prototypes_labels: torch.Tensor, name: str) -> None:
        """
        Dodaje rozwiązanie do listy rozwiązań.
        """
        self.sol_list.append({
            "name": name,
            "prototypes": prototypes.to(DEVICE),
            "prototypes_labels": prototypes_labels.to(DEVICE),
        })

    def points_from_accuracy(self, acc: float) -> int:
        """
        Przelicza dokładność na liczbę punktów (0–100).
        """
        result = (acc - 0.4) / (0.68 - 0.4) * 100
        result = int(round(result))
        return min(100, max(0, result))

    def _print_one_solution(
        self,
        prototypes: torch.Tensor,
        prototypes_labels: torch.Tensor,
        name: str
    ) -> None:
        """
        Wyświetla dokładność i liczbę punktów dla jednego rozwiązania.
        """
        acc_val = accuracy(prototypes, prototypes_labels, self.val_embeddings, self.val_labels)
        points = self.points_from_accuracy(acc_val)
        print(f"{name} ({prototypes.shape[0]} prototypes) Validation Accuracy: {acc_val * 100:.2f}% | (Points {points})")

    def print_solutions(self) -> None:
        """
        Wyświetla wyniki dla wszystkich zarejestrowanych rozwiązań.
        """
        for sol_dict in self.sol_list:
            name = sol_dict['name']
            prototypes = sol_dict['prototypes']
            prototypes_labels = sol_dict['prototypes_labels']
            
            self._print_one_solution(
                prototypes,
                prototypes_labels,
                name
            )

# Twoje Rozwiązanie
W tej sekcji należy umieścić Twoje rozwiązanie. Wprowadzaj zmiany wyłącznie tutaj!

Aktualne rozwiązanie polega na losowaniu prototypów ze zbioru treningowego.
Twoim zadaniem jest zmodyfikowanie klasy `YourSolution`. 

Twoja klasa `YourSolution` powinna posiadać metodę:

```python
def get_prototypes(self, train_embeddings, train_labels) -> Tuple[torch.Tensor, torch.Tensor]:
    ...
    return prototypes, prototypes_labels
```

Metoda ta powinna zwracać krotkę dwóch obiektów typu torch.Tensor.

- Pierwszy tensor, nazwany `prototypes`, powinien mieć kształt `[150, D]`, gdzie `D` to wymiar zanurzeń (ang. embeddings).

    `prototypes.device` powinien być równy stałej `DEVICE`, zdefiniowanej poniżej.

    `prototypes.dtype` powinien być `torch.float32`.

- Drugi tensor, `prototypes_labels`:

    Powinien mieć kształt `[150]`.

    Typ danych: `torch.long`.

    Każda wartość powinna należeć do zakresu `0 <= class < 100`.

Te wymagania będą sprawdzane przez klasę SolutionHolder.

In [12]:
# Twoje importy ...

In [ ]:
class YourSolution(): 
    def get_random_150_samples(self, unique_labels: np.ndarray) -> np.ndarray:
        """
        Przydziela losowo 1–2 próbki na klasę, łącznie 150 próbek.
        """
        num_samples_per_class = np.ones_like(unique_labels)
        double_samples = np.random.choice(
            num_samples_per_class.shape[0],
            50,
            replace = False
        )
        num_samples_per_class[double_samples] = num_samples_per_class[double_samples] + 1

        return num_samples_per_class

    def get_random_samples_per_class(
        self,
        embeddings: torch.Tensor,
        labels: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Wybiera losowo próbki embeddingów z każdej klasy na podstawie rozkładu.
        """
        embeddings = embeddings.numpy(force=True)
        labels = labels.numpy(force=True)

        unique_labels = np.unique(labels)
        num_samples_per_class = self.get_random_150_samples(unique_labels)

        random_samples = []
        for i, label in enumerate(unique_labels):
            class_indices = np.where(labels == label)[0]
            selected_indices = np.random.choice(class_indices, size=num_samples_per_class[i], replace=False)
            random_samples.extend(selected_indices)

        return torch.tensor(embeddings[random_samples], device=DEVICE), \
                torch.tensor(labels[random_samples], device=DEVICE)
        
    def get_prototypes(
        self,
        train_embeddings: torch.Tensor,
        train_labels: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Zwraca 150 losowych prototypów i odpowiadające im etykiety.

        [WAŻNE] Pamiętaj, aby zaimplementować w swojej klasie metodę `get_prototypes`
        tak jak opisano w sekcji `Ograniczenia`. Ta metoda może Ci służyć jako wzór. 
        """
        prototypes, prototypes_labels = self.get_random_samples_per_class(
            train_embeddings, train_labels
        )
        prototypes = prototypes.to(DEVICE)
        prototypes_labels = prototypes_labels.to(DEVICE)
        return prototypes, prototypes_labels

# Ewaluacja
Uruchomienie poniższych komórek pozwoli sprawdzić, ile punktów zdobyłoby Twoje rozwiązanie na danych walidacyjnych. Przed wysłaniem upewnij się, że cały notebook (również z ustawioną flagą FINAL_EVALUATION_MODE = True) wykonuje się od początku do końca bez błędów i bez konieczności ingerencji użytkownika po wybraniu opcji "Run All".

In [14]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:   
    solution_holder = SolutionHolder(
        train_embeddings=train_embeddings,
        train_labels=train_labels,
        val_embeddings=val_embeddings,
        val_labels=val_labels,
    )

    solution_holder.register_class_solution(YourSolution, "Your solution")
    solution_holder.print_solutions()

Your solution (150 prototypes) Validation Accuracy: 18.34% | (Points 0)


Możesz rejestrować różne swoje rozwiązania do oceny za pomocą poniższej komórki.

In [15]:
if not FINAL_EVALUATION_MODE:   
    solution_holder.register_class_solution(YourSolution, "Different solution")
    solution_holder.print_solutions()

Your solution (150 prototypes) Validation Accuracy: 18.34% | (Points 0)
Different solution (150 prototypes) Validation Accuracy: 17.97% | (Points 0)


Ponieżej znajduje się komorka, która automatycznie zapisze twoje prototypy na sprawdzarce, aby mogła użyć ich do klasyfikacji zbioru testowego. 

In [16]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if FINAL_EVALUATION_MODE: 
    solution = YourSolution()
    prototypes, prototypes_labels = solution.get_prototypes(
        train_embeddings, 
        train_labels
    )
     
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    torch.save(prototypes, os.path.join(OUTPUT_PATH, "prototypes.pt"),)
    torch.save(prototypes_labels, os.path.join(OUTPUT_PATH, "prototypes_labels.pt"))